In [1]:
import re
import psycopg2

hostname = 'localhost'
database = 'Counts'
username = 'postgres'
pwd = 1111
port_id = 5432

try:
    conn = psycopg2.connect(host=hostname, dbname=database, user=username, password=pwd, port=port_id)
    cur = conn.cursor()

    cur.execute("DROP TABLE IF EXISTS Counts")
    
    cur.execute("""
    CREATE TABLE Counts (
        org TEXT PRIMARY KEY,
        count INTEGER
    );
    """)

    filename = input("Enter file name: ")
    if len(filename) < 1: 
        filename = 'mbox-short.txt'

    with open(filename) as fh:
        for line in fh:
            if not line.startswith('From: '): 
                continue
            all_org = re.findall(r"@(\S+)", line)
            if all_org:  
                org = all_org[0].strip()
                cur.execute("SELECT count FROM Counts WHERE org = %s", (org,))
                row = cur.fetchone()
                if row is None:
                    cur.execute("INSERT INTO Counts (org, count) VALUES (%s, 1)", (org,))
                else:
                    cur.execute("UPDATE Counts SET count = count + 1 WHERE org = %s", (org,))

    conn.commit()

    sqlstr = "SELECT org, count FROM Counts ORDER BY count DESC LIMIT 10"
    cur.execute(sqlstr)
    rows = cur.fetchall()

    for row in rows:
        print(str(row[0]), row[1])

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    if cur:
        cur.close()
    if conn:
        conn.close()


Enter file name: mbox.txt
iupui.edu 536
umich.edu 491
indiana.edu 178
caret.cam.ac.uk 157
vt.edu 110
uct.ac.za 96
media.berkeley.edu 56
ufp.pt 28
gmail.com 25
whitman.edu 17
